<a href="https://colab.research.google.com/github/gdoteof/neuralnet_stuff/blob/master/kaggle_whales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install fast.ai dependencies.  

Here you want to make sure you have the gpu runtime selected.

![alt text](https://i.imgur.com/tCvtjwC.png)

The below will download and run a script to install fast.ai dependencies.

In [3]:
  !curl https://raw.githubusercontent.com/fastai/course-v3/master/docs/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   665  100   665    0     0   1973      0 --:--:-- --:--:-- --:--:--  1973
    100% |████████████████████████████████| 5.7MB 1.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x63770000 @  0x7f36d28c12a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
Cloning into 'course-v3'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40

In [0]:
import sys
from fastai import *
from fastai.vision import *

###Check GPU

The following should output:


```
1.0.0
True
True```



In [5]:

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

1.0.0
True
True


###(Optional) Connect Google Drive

Go the the URL, give permissions on your google account, and copy the code in the box that is presented when you run the following code.

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


###Download data from Kaggle

Generate an API Token from Kaggle.  They will give you a file called kaggle.json.  Inside that file there are two values you need to fill in for `KAGGLE_USERNAME` and `KAGGLE_KEY`

In [0]:
!KAGGLE_USERNAME=gdoteof KAGGLE_KEY=1xxxxxxxxxxxxxxxxxxxxxxxxxx2 kaggle competitions download -c humpback-whale-identification


Alright, now let's move it all to our google drive.  

In [0]:
BASE = "/content/drive/My Drive/ai/whales"

In [0]:
!mkdir -p "{BASE}/train"

In [0]:
!mv test.zip train.zip train.csv "{BASE}"

In [0]:
!echo {BASE}

/content/drive/My Drive/ai/whales


In [0]:
!unzip "{BASE}"/train.zip -d "{BASE}/train"

In [0]:
!mkdir -p "{BASE}/test"

In [0]:
!unzip "{BASE}"/test.zip -d "{BASE}/test"

##Build our databunch!

First we set up some meta-parameters (size of the input image to the net and the batch size)

In [0]:
!ls "{BASE}"

models	test  test.zip	train  train.csv  train.zip


In [0]:
#ws means "with separation".  
#we are running into an issue where there are classes in the validation set but not training.
#this was going unnoticed before, but checks later are catching it
from fastai.torch_core import *
from fastai.basic_data import *
from fastai.data_block import *


def create_from_ll(lls:LabelLists, bs:int=64, ds_tfms:Optional[TfmList]=None,
            num_workers:int=defaults.cpus, tfms:Optional[Collection[Callable]]=None, device:torch.device=None,
            test:Optional[PathOrStr]=None, collate_fn:Callable=data_collate, size:int=None, no_check:bool=False, **kwargs)->'ImageDataBunch':
    "Create an `ImageDataBunch` from `LabelLists` `lls` with potential `ds_tfms`."
    lls = lls.transform(tfms=ds_tfms, size=size, **kwargs)
    if test is not None: lls.add_test_folder(test)
    return lls.databunch(bs=bs, tfms=tfms, num_workers=num_workers, collate_fn=collate_fn, device=device, no_check=no_check)

 
  
def from_df_ws(path:PathOrStr, df:pd.DataFrame, folder:PathOrStr='.', sep=None, valid_pct:float=0.2,
                fn_col:IntsOrStrs=0, label_col:IntsOrStrs=1, suffix:str='',
                **kwargs:Any)->'ImageDataBunch':
  "Create from a `DataFrame` `df`."
  
  msk = np.random.rand(len(df)) < 1 - valid_pct

  df_train = df[msk]
  df_valid = df[~msk]

  df_diff  = df_valid[~df_valid["Id"].isin(df_train["Id"])]
  
  df_valid = df_valid[~df_valid["Id"].isin(df_diff["Id"])]

  train_iil = ImageItemList.from_df(df_train, path=path, folder=folder, suffix=suffix, cols=fn_col)
  valid_iil = ImageItemList.from_df(df_valid, path=path, folder=folder, suffix=suffix, cols=fn_col)
  


  src = (ItemLists(path, train_iil, valid_iil)
            .label_from_df(sep=sep, cols=label_col)) 

  return ImageDataBunch.create_from_ll(src, **kwargs)


def from_csv_ws(path:PathOrStr, folder:PathOrStr='.', sep=None, csv_labels:PathOrStr='labels.csv', valid_pct:float=0.2,
            fn_col:int=0, label_col:int=1, suffix:str='',
            header:Optional[Union[int,str]]='infer', **kwargs:Any)->'ImageDataBunch':
        "Create from a csv file in `path/csv_labels`."
        path = Path(path)
        df = pd.read_csv(path/csv_labels, header=header)
        return from_df_ws(path, df, folder=folder, sep=sep, valid_pct=valid_pct,
                fn_col=fn_col, label_col=label_col, suffix=suffix, **kwargs)
      


In [0]:
sz=128
bs=512  #6.1gb usage
tfms = get_transforms(max_rotate=20, max_zoom=1.3, max_lighting=0.4, max_warp=0.4, p_affine=1., p_lighting=1.)
data = from_csv_ws(path=BASE, folder=f'train', csv_labels="train.csv", ds_tfms=tfms, bs=bs, size=sz)

In [0]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [0]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,error_rate


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fc7d56086a0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 713, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7fc7db9981d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 717, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", 

In [0]:
learn.save('whale-checkpoint')

In [0]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,error_rate
1,5.213348,4.353013,0.505915


In [0]:
learn.save('whale-checkpoin2')